In [1]:
import numpy as np
from common.functions import *
from common.gradient import numerical_gradient
from util.layers import *
from collections import OrderedDict
from util.functions import *

In [2]:
class FourLayerNet :
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size, weight_init_std = 0.01 ):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size1)
        self.params['b1'] = np.zeros(hidden_size1) # 은닉층1
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size1, hidden_size2)
        self.params['b2'] = np.zeros(hidden_size2) # 은닉층2
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size2, hidden_size3)
        self.params['b3'] = np.zeros(hidden_size3) # 은닉층3
        self.params['W4'] = weight_init_std * np.random.randn(hidden_size3, output_size)
        self.params['b4'] = np.zeros(output_size) # 출력층

        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['LeakyReLU'] = LeakyReLU()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['ELU1'] = ELU()
        self.layers['Affine3'] = Affine(self.params['W3'], self.params['b3'])
        self.layers['ELU2'] = ELU()
        self.layers['Affine4'] = Affine(self.params['W4'], self.params['b4'])

        self.lastLayers = SoftmaxWithLoss()

    def predict (self, x) :
        for layer in self.layers.values() :
            x = layer.forward(x) # 예측 단계에서는 softmax함수가 어차피 실제값과 같기 때문에 필요없음.(argmax)
                                # 단, 학습단계에서는 loss값을 구해야 하기 때문에 softmax적용이 필요.

        return x

    def loss(self, x, t) :
        y = self.predict(x)

        return self.lastLayers.forward(y,t)

    def accuracy(self, x, t) :
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / x.shape[0]
        return accuracy
    
    
    def numerical_gradient(self, x, t):
        loss_W = lambda _: self.loss(x,t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    

    def gradient(self, x, t) :
        
        self.loss(x,t)

        dout = 1
        dout = self.lastLayers.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers :
            dout = layer.backward(dout)

        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        grads['W3'] = self.layers['Affine3'].dW
        grads['b3'] = self.layers['Affine3'].db
        grads['W4'] = self.layers['Affine4'].dW
        grads['b4'] = self.layers['Affine4'].db

        return grads


In [3]:
from util.mnist import load_mnist

In [60]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize= True, one_hot_label = True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

In [61]:
print(x_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [62]:
for key in grad_numerical.keys() :
    print(key, grad_backprop[key].shape, grad_numerical[key].shape)
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ':' + str(diff))

W1 (784, 50) (784, 50)
W1:2.23288841097496e-13
b1 (50,) (50,)
b1:7.732881072671297e-13
W2 (50, 10) (50, 10)
W2:8.917700797641051e-13
b2 (10,) (10,)
b2:1.2101430968414206e-10


In [4]:
from tqdm import tqdm

In [5]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize= True, one_hot_label = True)

network = FourLayerNet(input_size=784, hidden_size1=500, hidden_size2 = 300, hidden_size3 = 100, output_size=10)

train_loss_list = []
# 추가
train_acc_list = []
test_acc_list = []

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

#추가
iter_per_epoch = train_size / batch_size #정하기 나름! (ex :1000)




for i in tqdm(range(iters_num)) :
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.gradient(x_batch, t_batch)

    for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3', 'W4', 'b4') :
        network.params[key] -= learning_rate * grad[key]  #!!!학습!!!

    loss = network.loss(x_batch,t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0 :
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc |" + str(train_acc) + ", "+ str(test_acc))

  0%|          | 10/10000 [00:01<20:19,  8.19it/s] 

train acc, test acc |0.0993, 0.1032


  6%|▌         | 613/10000 [00:09<06:18, 24.77it/s]

train acc, test acc |0.24883333333333332, 0.246


 12%|█▏        | 1218/10000 [00:17<06:04, 24.08it/s]

...

 90%|█████████ | 9010/10000 [01:57<00:38, 26.03it/s] 

train acc, test acc |0.9973666666666666, 0.9778


 96%|█████████▌| 9614/10000 [02:05<00:15, 25.14it/s]

train acc, test acc |0.99555, 0.9744


100%|██████████| 10000/10000 [02:09<00:00, 77.24it/s]


In [8]:
print(max(train_acc_list), max(test_acc_list))

0.9973666666666666 0.9778
